In [ ]:
import numpy as np
import pandas as pd
import mdtraj
import pickle
import matplotlib.pyplot as plt
import os
import scipy.stats as ss
import random
from collections import Counter
import seaborn as sns
from statsmodels.stats import weightstats

plt.style.use('seaborn')

In [ ]:
def save_obj(obj, name ):
    with open('/Users/h.xia/Desktop/Griffith_Lab/R_shiny_visualization/anchor_analysis/obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('/Users/h.xia/Desktop/Griffith_Lab/R_shiny_visualization/anchor_analysis/obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
random_subset = load_obj('random_subset_pMHC_crystallography_analysis_2')
sample_data_with_binding = load_obj('pMHC_v2_validation_sample_data_with_binding')

In [ ]:
sample_data_with_binding_subset = sample_data_with_binding.iloc[random_subset]

In [ ]:
print(ss.ttest_ind(sample_data_with_binding_subset['Distance'], sample_data_with_binding_subset['Random Distance'], equal_var=False))
print(ss.ttest_ind(sample_data_with_binding_subset['SASA'], sample_data_with_binding_subset['Random SASA'], equal_var=False))

In [ ]:
print(ss.ttest_ind(sample_data_with_binding['Distance'], sample_data_with_binding['Random Distance'], equal_var=False))
print(ss.ttest_ind(sample_data_with_binding['SASA'], sample_data_with_binding['Random SASA'], equal_var=False))

In [ ]:
## Testing for normal distribution
ss.shapiro(sample_data_with_binding_subset['Distance'] - sample_data_with_binding_subset['Random Distance'])

In [ ]:
## Testing for equal variance
ss.levene(sample_data_with_binding_subset['Distance'],
             sample_data_with_binding_subset['Random Distance'],
             center= 'mean')

## Repeat the shuffling test

In [ ]:
binding_data = pd.read_csv("./validation_analysis_pvacbind_output_combined.txt", delimiter='\t')
sample_data = pd.read_csv("pdb_files/cleaned_list_pdb_structures.tsv", delimiter='\t', header=None)

In [ ]:
SASA_results = load_obj('SASA_results_validation_overall_new')
Distance_with_percentile_results = load_obj('Distance_with_percentile_results_validation_overall_new')
all_anchor_overall_pos_score_dict = load_obj('all_anchor_overall_pos_score_dict_r4')

In [ ]:
for k in range(0,5):
    randomized_SASA = []
    randomized_Distance = []
    for n, i in sample_data_with_binding.iterrows():
        sasa_calculation = SASA_results[i['PDB_ID']]
        distance_calculation = Distance_with_percentile_results[i['PDB_ID']]
        prediction_calculation = all_anchor_overall_pos_score_dict["HLA-"+i['HLA Allele']][len(i['Peptide'])]
        random.shuffle(sasa_calculation)
        random.shuffle(distance_calculation)
        ran_corr1, ran_p1 = ss.spearmanr(sasa_calculation, prediction_calculation)
        ran_corr2, ran_p2 = ss.spearmanr(distance_calculation, prediction_calculation)
        randomized_SASA.append(ran_corr1)
        randomized_Distance.append(ran_corr2)
    
    col_name_sasa = 'Random SASA Round ' + str(k+1)
    col_name_distance = 'Random Distance Round ' + str(k+1)
    sample_data_with_binding[col_name_sasa] = randomized_SASA
    sample_data_with_binding[col_name_distance] = randomized_Distance

In [ ]:
sample_data_with_binding_subset = sample_data_with_binding.iloc[random_subset]

In [ ]:
for k in range(0,5):
    col_name_sasa = 'Random SASA Round ' + str(k+1)
    col_name_distance = 'Random Distance Round ' + str(k+1)
    print("Round "+str(k))
    print(ss.ttest_ind(sample_data_with_binding_subset['Distance'], sample_data_with_binding_subset[col_name_distance], equal_var=False))
    print(ss.ttest_ind(sample_data_with_binding_subset['SASA'], sample_data_with_binding_subset[col_name_sasa], equal_var=False))
    print(weightstats.ztest(sample_data_with_binding_subset['Distance'], sample_data_with_binding_subset[col_name_distance], value=0))
    print(weightstats.ztest(sample_data_with_binding_subset['SASA'], sample_data_with_binding_subset[col_name_sasa], value=0))

In [ ]:
print(weightstats.ztest(sample_data_with_binding_subset['Distance'], sample_data_with_binding_subset[col_name_distance], value=0))
print(weightstats.ztest(sample_data_with_binding_subset['SASA'], sample_data_with_binding_subset[col_name_sasa], value=0))

In [ ]:
sample_data_with_binding_subset.to_excel("Full sasa and distance dataset after randomly shuffling 5 times.xlsx")